In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import vae_ex
import vae_arch
import tensorflow as tf
import haiku as hk
import matplotlib.pyplot as plt
import herculens_util
import numpy as np
import sys
sys.path.insert(0, '/Users/smericks/Desktop/CS236/sl_project/')
import visualization_utils

In [ ]:
with open('model_lensing_epoch017.pckl', 'rb') as file:
    params, opt_state = pickle.load(file)

In [ ]:
# can control number of epochs here!! 
val_dataset = vae_ex.generate_tf_dataset('COSMOS_23.5_training_sample/custom_cutouts_round2/validation_data.tfrecord',
    1000,1)

for batch_v in val_dataset:
    batch_v = tf.squeeze(batch_v)
    val_images = batch_v.numpy()
    break

In [ ]:
loss_df = np.loadtxt('loss.csv',delimiter=',')
plt.plot(loss_df)
#plt.yscale('log')
#plt.ylim([-38000,-37000])

In [ ]:
rng_seq = hk.PRNGSequence(42)
outputs = vae_arch.model.apply(params, next(rng_seq), val_images)

In [ ]:
import matplotlib.colors as colors

i = 0
coords = [-.5,0.,.5]
fig1,axs1 = plt.subplots(3,10,figsize=(10,3))
plt.subplots_adjust(wspace=None, hspace=None)
fig2,axs2 = plt.subplots(3,10,figsize=(10,3))
plt.subplots_adjust(wspace=None, hspace=None)
for i in range(0,10):
    im1 = val_images[i]
    im2 = outputs.image[i]
    axs1[0,i].imshow(im1)
    axs1[0,i].get_xaxis().set_ticks([])
    axs1[0,i].get_yaxis().set_ticks([])
    #axs1[0,0].set_title('original galaxy',fontsize=10)
    axs1[1,i].imshow(im2)
    axs1[1,i].get_xaxis().set_ticks([])
    axs1[1,i].get_yaxis().set_ticks([])
    #axs1[0,1].set_title('decoded galaxy',fontsize=10)
    resid_norm = colors.TwoSlopeNorm(vmin=-0.001,vcenter=0,vmax=.001)
    axs1[2,i].imshow(im1-im2,norm=resid_norm,cmap='bwr')
    axs1[2,i].get_xaxis().set_ticks([])
    axs1[2,i].get_yaxis().set_ticks([])
    #axs1[0,2].set_title('residual',fontsize=10)
    if i == 0:
        axs1[0,i].set_ylabel('original')
        axs1[1,i].set_ylabel('decoded')
        axs1[2,i].set_ylabel('residual')


    print(np.median(im1))
    print(np.std(im1))
    print(np.max(im1))
    lens_params = np.array([1.2,0.,0.,0.0,0.])
    im1,im2 = herculens_util.apply_lensing(im1,im2,lens_params)
    print(np.median(im1))
    print(np.std(im1))
    print(np.max(im1))
    axs2[0,i].imshow(im1,norm='asinh')
    axs2[0,i].get_xaxis().set_ticks([])
    axs2[0,i].get_yaxis().set_ticks([])
    #axs2[i,0].set_title('lensed original',fontsize=10)
    axs2[1,i].imshow(im2,norm='asinh')
    axs2[1,i].get_xaxis().set_ticks([])
    axs2[1,i].get_yaxis().set_ticks([])
    #axs2[i,1].set_title('lensed decoded',fontsize=10)
    resid_norm = colors.TwoSlopeNorm(vmin=-0.0001,vcenter=0,vmax=.0001)
    axs2[2,i].imshow(im1-im2,norm=resid_norm,cmap='bwr')
    axs2[2,i].get_xaxis().set_ticks([])
    axs2[2,i].get_yaxis().set_ticks([])
    if i == 0:
        axs2[0,i].set_ylabel('original')
        axs2[1,i].set_ylabel('decoded')
        axs2[2,i].set_ylabel('residual')
    #axs2[i,2].set_title('residual',fontsize=10)

In [ ]:
files = ['model_test_epoch010.pckl','model_test_epoch032.pckl','model_lensing_epoch017.pckl']
rng_seq = hk.PRNGSequence(42)
outputs_list = []
for f in files:
    with open(f, 'rb') as file:
        params, opt_state = pickle.load(file)
    outputs = vae_arch.model.apply(params, next(rng_seq), val_images)
    outputs_list.append(np.asarray(outputs.mean))

param_labels = ['$z_0$','$z_1$','$z_2$','$z_3$','$z_4$','$z_5$','$z_6$','$z_7$','$z_8$','$z_9$']
visualization_utils.overlay_contours(outputs_list,['grey','blue','green'],[0,1,2,3,4,5,6,7,8,9],
    param_labels,['epoch 10','regular objective','strong lensing objective'],
    true_params=None,save_path=None,bounds=None,
    burnin=int(0),annotate_medians=False,dpi=200)

In [ ]:
z_means = outputs.mean

print(z_means.shape)

plt.scatter(z_means[:,0],z_means[:,1],c=np.sum(val_images,axis=(1,2)))
plt.xlabel('$z_0$')
plt.ylabel('$z_1$')
cbar = plt.colorbar()
cbar.ax.set_yticklabels([])
cbar.set_label('total flux', rotation=270)
plt.title('Epoch 1')

plt.figure()
plt.scatter(z_means[:,2],z_means[:,3],c=np.sum(val_images,axis=(1,2)))
plt.xlabel('$z_2$')
plt.ylabel('$z_3$')
plt.colorbar()
plt.title('Epoch 1')

plt.figure()
plt.scatter(z_means[:,4],z_means[:,5],c=np.sum(val_images,axis=(1,2)))
plt.xlabel('$z_4$')
plt.ylabel('$z_5$')
plt.colorbar()
plt.title('Epoch 1')

In [ ]:
rng_seq = hk.PRNGSequence(42)
outputs = vae_ex.model.apply(params, next(rng_seq), val_images)

outputs.image[0]

fig,axs = plt.subplots(1,2)

axs[0].matshow(val_images[0])
axs[1].matshow(outputs.image[0])

print(np.std(val_images[0]))
print(np.max(val_images[0]))
print(np.std(outputs.image[0]))
print(np.max(outputs.image[0]))|